In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble

Задача: Разработать модель, которая способна предсказывать летальность госпитализированных пациентов в зависимости 
от различных причин госпитализации

In [3]:
# Считываем данные
fileway = 'C:\\Users\\HOME\\Desktop\\dataset.csv'
data = pd.read_csv(fileway, sep = ',')

--- Вычисляем основные характеристики ---

In [4]:
# Среднее, стандартное отклонение и другие
data.describe(include = None)

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,...,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
count,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90379.000000,91713.000000,91713.000000,88993.000000,...,83766.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,91713.000000
mean,65606.079280,65537.131464,105.669262,62.309516,29.185818,0.183736,169.641588,508.357692,0.835766,84.028340,...,0.043955,0.000857,0.015693,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638,0.086302
std,37795.088538,37811.252183,62.854406,16.775119,8.275142,0.387271,10.795378,228.989661,2.487756,25.011497,...,0.217341,0.029265,0.124284,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169,0.280811
min,1.000000,1.000000,2.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-24.947222,38.600000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32852.000000,32830.000000,47.000000,52.000000,23.641975,0.000000,162.500000,369.000000,0.035417,66.800000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,65665.000000,65413.000000,109.000000,65.000000,27.654655,0.000000,170.100000,504.000000,0.138889,80.300000,...,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,98342.000000,98298.000000,161.000000,75.000000,32.930206,0.000000,177.800000,679.000000,0.409028,97.100000,...,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,131051.000000,131051.000000,204.000000,89.000000,67.814990,1.000000,195.590000,927.000000,159.090972,186.000000,...,0.970000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# Корреляция
pd.set_option('display.max_columns', None)
data.corr(numeric_only = True)

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,heart_rate_apache,intubated_apache,map_apache,resprate_apache,temp_apache,ventilated_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_glucose_min,d1_potassium_max,d1_potassium_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
encounter_id,1.000000,-0.009575,-0.004532,-0.003990,-0.001068,0.002036,-0.005400,-0.000992,-0.000570,-0.004376,0.000113,-0.000651,0.001138,0.007920,0.003276,0.007664,0.002400,0.006516,-0.001718,-0.003172,0.000759,0.006498,0.005162,-0.005775,-0.002547,0.000802,-0.002229,0.001202,-0.003891,-0.002045,-0.000332,-0.000553,0.000734,-0.000472,0.006880,-0.000296,0.000041,0.000512,-0.000484,-0.002455,-0.000063,-0.002115,-0.004533,0.003035,-0.001529,0.003506,0.000586,0.005045,-0.000167,-0.000930,-0.000261,0.002202,0.000654,0.002838,0.002682,0.000351,-0.001577,-0.004182,0.002714,0.000849,0.002974,0.001240,0.003672,0.002068,-0.001755,-0.000521,-0.000513,0.000344,0.001907,0.007601,0.003402,-0.000972,-0.001707,-0.002172,-0.001176,-0.004646,-0.005243
patient_id,-0.009575,1.000000,-0.007075,0.006343,-0.001380,0.001387,0.002902,-0.001770,-0.004412,0.000275,-0.001539,0.004215,0.002260,0.003980,0.001551,0.002216,-0.006673,0.000471,0.004735,0.001303,-0.002413,0.000179,0.002535,-0.004433,-0.003287,-0.004180,-0.002946,-0.004605,0.001330,0.001570,-0.004471,-0.003249,-0.004160,-0.003740,0.003159,0.002041,-0.002040,-0.002318,-0.000021,-0.006877,0.000188,-0.007695,0.000807,0.003264,-0.006719,-0.008462,-0.007044,-0.011464,0.000688,0.001328,-0.005861,-0.008053,-0.006467,-0.010379,0.004802,0.001478,0.001787,0.000198,-0.003579,-0.006587,-0.004689,-0.009214,-0.003179,0.003235,0.000674,0.001502,0.003735,0.001625,-0.002426,0.001307,0.000434,-0.001667,0.000548,0.000844,0.002333,-0.003065,0.004877
hospital_id,-0.004532,-0.007075,1.000000,-0.008673,0.012874,0.052123,0.027895,0.004526,-0.001285,0.026314,0.006806,0.031896,0.053985,0.000844,-0.011727,-0.020512,-0.017576,-0.002954,-0.007649,0.027670,0.001011,-0.023262,-0.033490,0.030761,-0.030503,0.020925,-0.028594,0.022370,-0.012901,-0.005449,-0.027288,0.016304,-0.028030,0.018439,-0.042902,-0.005750,-0.001347,0.011384,-0.037497,0.014535,-0.037839,0.015300,0.008949,-0.039445,-0.002574,-0.010827,0.004452,-0.002286,-0.015428,-0.023639,0.000597,-0.010510,0.000894,-0.004106,-0.032078,-0.032271,0.018389,0.013308,-0.005402,-0.015266,-0.008606,-0.012545,-0.000811,0.016819,0.010070,-0.012084,-0.006993,-0.000283,-0.004947,0.002962,0.011978,0.001362,0.000146,-0.002992,0.002588,-0.004730,-0.001255
age,-0.003990,0.006343,-0.008673,1.000000,-0.087077,0.067320,-0.109937,-0.024257,0.049872,-0.127252,0.022914,-0.056060,0.059246,-0.001684,0.026363,0.025843,-0.007754,-0.017829,-0.149495,0.015396,-0.015870,0.037328,-0.082265,0.041296,-0.062958,-0.211598,-0.060418,-0.209291,-0.135417,-0.143705,0.006123,-0.131755,0.006198,-0.130206,0.032206,0.034439,-0.038621,-0.084488,0.107642,-0.060160,0.107016,-0.060322,-0.082764,-0.070002,-0.147122,-0.200129,-0.136832,-0.193331,-0.164788,-0.165454,-0.059301,-0.108958,-0.057717,-0.103732,0.02

In [6]:
# Характеристики категориальных признаков
data.describe(include = ['object'])

,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem
count,90318,91688,91601,91713,91713,90051,90051
unique,6,2,5,3,8,11,10
top,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular
freq,70684,49469,54060,86183,50586,29999,38816


--- Обрабатываем пропущенные значения ---

In [8]:
# Считаем количество пропущенных значений
data.isnull().sum()

encounter_id                      0
patient_id                        0
hospital_id                       0
age                            4228
bmi                            3429
                               ... 
lymphoma                        715
solid_tumor_with_metastasis     715
apache_3j_bodysystem           1662
apache_2_bodysystem            1662
hospital_death                    0
Length: 84, dtype: int64

In [9]:
# Удаляем строки, в которых заполнено меньше 90% данных
data = data.dropna(axis = 0, thresh = data.shape[1] * 0.9)
data = data.reset_index()
data = data.drop('index', axis = 1)
data

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,heart_rate_apache,intubated_apache,map_apache,resprate_apache,temp_apache,ventilated_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_glucose_min,d1_potassium_max,d1_potassium_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,66154,25312,118,68.0,22.730000,0,Caucasian,M,180.3,Floor,92,admit,CTICU,0.541667,73.9,113.0,502.01,0,0.0,3.0,6.0,0.0,4.0,118.0,0.0,40.0,36.0,39.3,0.0,68.0,37.0,68.0,37.0,119.0,72.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,131.0,73.0,131.0,73.0,39.9,37.2,68.0,63.0,68.0,63.0,119.0,108.0,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,131.0,115.0,131.0,115.0,168.0,109.0,4.0,3.4,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,114252,59342,81,77.0,27.420000,0,Caucasian,F,160.0,Floor,90,admit,Med-Surg ICU,0.927778,70.2,108.0,203.01,0,0.0,1.0,3.0,0.0,1.0,120.0,0.0,46.0,33.0,35.1,1.0,95.0,31.0,95.0,31.0,118.0,72.0,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,159.0,67.0,159.0,67.0,36.3,35.1,61.0,48.0,61.0,48.0,114.0,100.0,85.0,57.0,85.0,57.0,31.0,28.0,95.0,70.0,95.0,71.0,95.0,71.0,145.0,128.0,4.2,3.8,0.47,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,119783,50777,118,25.0,31.950000,0,Caucasian,F,172.7,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,95.3,122.0,703.03,0,0.0,3.0,6.0,0.0,5.0,102.0,0.0,68.0,37.0,36.7,0.0,88.0,48.0,88.0,48.0,96.0,68.0,102.0,68.0,102.0,68.0,21.0,8.0,98.0,91.0,148.0,105.0,148.0,105.0,37.0,36.7,88.0,58.0,88.0,58.0,96.0,78.0,91.0,83.0,91.0,83.0,20.0,16.0,98.0,91.0,148.0,124.0,148.0,124.0,NaN,NaN,NaN,NaN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
3,79267,46918,118,81.0,22.640000,1,Caucasian,F,165.1,Operating Room / Recovery,92,admit,CTICU,0.000694,61.7,203.0,1206.03,1,0.0,4.0,6.0,0.0,5.0,114.0,1.0,60.0,4.0,34.8,1.0,48.0,42.0,48.0,42.0,116.0,92.0,84.0,84.0,84.0,84.0,23.0,7.0,100.0,95.0,158.0,84.0,158.0,84.0,38.0,34.8,62.0,44.0,NaN,NaN,100.0,96.0,92.0,71.0,NaN,NaN,12.0,11.0,100.0,99.0,136.0,106.0,NaN,NaN,185.0,88.0,5.0,3.5,0.04,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,0
4,33181,74489,83,67.0,27.560000,0,Caucasian,M,190.5,Accident & Emergency,95,admit,Med-Surg ICU,0.000694,100.0,301.0,403.01,0,0.0,4.0,6.0,0.0,5.0,113.0,0.0,130.0,35.0,36.6,0.0,100.0,61.0,100.0,61.0,113.0,83.0,127.0,80.0,127.0,80.0,32.0,10.0,97.0,91.0,173.0,107.0,173.0,107.0,36.8,36.6,89.0,89.0,89.0,89.0,83.0,83.0,111.0,111.0,111.0,111.0,12.0,12.0,97.0,97.0,143.0,143.0,143.0,143.0,156.0,125.0,3.9,3.7,0.05,0.02,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86021,118430,83320,6,21.0,24.874059,0,Caucasian,M,185.4,Accident &

In [10]:
# Берём медиану для числовых значений
data_num = data.median(axis = 0, numeric_only = True)
data_num

encounter_id                   65736.500000
patient_id                     65452.500000
hospital_id                      112.000000
age                               65.000000
bmi                               27.616817
                                   ...     
immunosuppression                  0.000000
leukemia                           0.000000
lymphoma                           0.000000
solid_tumor_with_metastasis        0.000000
hospital_death                     0.000000
Length: 77, dtype: float64

In [11]:
# Берём популярные значения для категориальных признаков
data_categ = data.describe(include = ['object']).loc['top']
data_categ

ethnicity                          Caucasian
gender                                     M
icu_admit_source        Accident & Emergency
icu_stay_type                          admit
icu_type                        Med-Surg ICU
apache_3j_bodysystem          Cardiovascular
apache_2_bodysystem           Cardiovascular
Name: top, dtype: object

In [12]:
# Заполняем этими данными пустые места
data = data.fillna(data_num, axis = 0)
data = data.fillna(data_categ, axis = 0)

In [13]:
# Проверяем, что теперь пропущенных значений не осталось
data.isnull().sum()

encounter_id                   0
patient_id                     0
hospital_id                    0
age                            0
bmi                            0
                              ..
lymphoma                       0
solid_tumor_with_metastasis    0
apache_3j_bodysystem           0
apache_2_bodysystem            0
hospital_death                 0
Length: 84, dtype: int64

--- Обрабатываем категориальные признаки ---

In [15]:
# Разделяем столбцы на категориальные и числовые
categ_col = [c for c in data.columns if data[c].dtype.name == 'object']
num_col = [c for c in data.columns if data[c].dtype.name != 'object']

# Разделяем категориальные признаки на бинарные и небинарные
binary_col = [c for c in categ_col if data[c].unique().size == 2]
nonbinary_col = [c for c in categ_col if data[c].unique().size > 2]

print(binary_col)
print(nonbinary_col)

['gender']
['ethnicity', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']


In [16]:
# Работаем с бинарным признаком. Заменяем текстовые значения нулями и единичками
data['gender'].unique()

array(['M', 'F'], dtype=object)

In [17]:
data = data.replace({'gender': {'M': 0, 'F': 1}})
data['gender'].describe()

count    86026.000000
mean         0.460919
std          0.498473
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: gender, dtype: float64

In [18]:
# Работаем с небинарными признаками. Разделяем каждый признак с s значениями на s признаков со значениями 0 и 1
data_nonbin = pd.get_dummies(data[nonbinary_col])
data_nonbin.describe()

,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,icu_admit_source_Accident & Emergency,icu_admit_source_Floor,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,apache_3j_bodysystem_Cardiovascular,apache_3j_bodysystem_Gastrointestinal,apache_3j_bodysystem_Genitourinary,apache_3j_bodysystem_Gynecological,apache_3j_bodysystem_Hematological,apache_3j_bodysystem_Metabolic,apache_3j_bodysystem_Musculoskeletal/Skin,apache_3j_bodysystem_Neurological,apache_3j_bodysystem_Respiratory,apache_3j_bodysystem_Sepsis,apache_3j_bodysystem_Trauma,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
count,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.00000,86026.000000,86026.000000,86026.000000,86026.000000
mean,0.103643,0.012333,0.784693,0.042243,0.008788,0.048299,0.597773,0.170460,0.199033,0.026213,0.006521,0.940936,0.005556,0.053507,0.077558,0.050206,0.038605,0.051519,0.087229,0.552310,0.084335,0.058238,0.331470,0.099807,0.024330,0.003464,0.007044,0.085067,0.012764,0.132065,0.128694,0.132320,0.042975,0.434369,0.099807,0.007044,0.085067,0.132065,0.02755,0.128694,0.042975,0.003731,0.038698
std,0.304799,0.110370,0.411038,0.201144,0.093332,0.214399,0.490350,0.376039,0.399275,0.159769,0.080491,0.235745,0.074335,0.225044,0.267476,0.218371,0.192652,0.221056,0.282173,0.497259,0.277891,0.234195,0.470744,0.299744,0.154072,0.058755,0.083635,0.278983,0.112253,0.338563,0.334863,0.338841,0.202803,0.495677,0.299744,0.083635,0.278983,0.338563,0.16368,0.334863,0.202803,0.060972,0.192874
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

--- Нормализация ---

In [19]:
# Текущие числовые характеристики. Убрали столбец 'hospital_death', так как это класс, а не признак

data_num = data[num_col]
data_num = data_num.drop('hospital_death', axis = 1)
data_num.describe()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,heart_rate_apache,intubated_apache,map_apache,resprate_apache,temp_apache,ventilated_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_glucose_min,d1_potassium_max,d1_potassium_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.00000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.00000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000
mean,65612.543115,65562.782356,106.118883,62.408644,29.105353,0.179155,169.632226,506.741671,0.825602,83.831610,184.08659,553.442177,0.196534,0.027910,3.469811,5.478576,0.008556,4.002104,99.865703,0.149525,88.175796,25.902554,36.416177,0.323542,88.745961,50.102434,88.744438,50.130844,103.160103,70.315521,104.752831,64.800130,104.690524,64.817753,28.909678,12.888406,99.255702,90.474380,148.494792,96.793121,148.428928,96.820737,37.281670,36.269388,75.465080,62.918188,75.757225,63.217051,92.321112,83.713633,91.583266,79.435380,91.496524,79.652466,22.640585,17.214551,98.066143,95.183735,133.237184,116.397333,132.961454,116.470195,173.056285,113.947376,4.242947,3.931282,0.090614,0.04783,0.000895,0.016042,0.225722,0.013287,0.026690,0.007242,0.004196,0.021040
std,37798.685560,37786.063465,62.799466,16.481279,8.135030,0.383484,10.730064,227.997921,2.434423,24.665788,85.84442,460.202782,0.397379,0.164716,0.947013,1.277198,0.092100,1.554918,30.942034,0.356607,42.210302,15.091049,0.821914,0.467830,19.732648,13.290854,19.738836,13.304738,22.049159,17.064611,20.746733,15.663139,20.632849,15.632166,10.675647,5.027784,1.777028,9.943749,25.710214,20.606472,25.749360,20.619637,0.686584,0.735744,18.416522,16.374862,18.188926,16.162435,21.845942,20.287866,20.401324,19.031621,19.998344,18.720302,7.447099,6.015851,3.096821,6.534302,27.566708,26.533141,27.227010,26.191027,84.308541,37.116208,0.633367,0.550635,0.225694,0.19644,0.029905,0.125636,0.418060,0.114500,0.161176,0.084792,0.064644,0.143519
min,1.000000,1.000000,2.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-0.250000,38.600000,101.00000,0.010000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,30.000000,0.000000,40.000000,4.000000,32.100000,0.000000,46.000000,13.000000,46.000000,13.000000,58.000000,0.000000,6

In [20]:
# Нормализуем числовые данные. Приводим их к в виду, когда среднее нулевое, а стандартное отклонение единичное
data_num = (data_num - data_num.mean()) / data_num.std()
data_num.describe()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,heart_rate_apache,intubated_apache,map_apache,resprate_apache,temp_apache,ventilated_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_glucose_min,d1_potassium_max,d1_potassium_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04
mean,1.610627e-16,4.935128e-17,-1.070448e-16,4.889700e-17,1.321540e-18,-9.250783e-17,-1.074412e-15,-4.228929e-17,-6.739856e-17,3.171697e-17,-1.400833e-16,-2.349038e-16,1.321540e-17,1.784080e-17,-2.537358e-16,2.385380e-16,2.775235e-17,-2.016175e-16,-3.237774e-17,6.607702e-18,6.285577e-17,-4.955777e-17,1.379688e-15,-8.953436e-17,-1.982311e-17,-9.779399e-17,1.744433e-16,1.955880e-16,2.828097e-16,-1.182779e-16,1.004371e-16,9.515091e-17,-2.960251e-16,4.466807e-16,-6.343394e-17,6.079086e-17,2.866421e-15,-3.303851e-16,1.037409e-16,2.405204e-16,1.420656e-16,2.722373e-16,7.962281e-15,-1.650604e-15,-2.527446e-17,-6.872010e-17,8.028358e-17,1.929449e-16,-1.229033e-16,-2.230099e-17,-3.449221e-16,-2.775235e-16,-1.235640e-16,-1.850157e-17,8.722167e-17,4.228929e-17,5.068108e-16,-8.636267e-16,4.826926e-16,-4.493237e-17,1.625495e-16,-1.612279e-16,-1.817118e-18,1.552810e-16,-7.813608e-17,-7.892900e-16,-8.788244e-17,2.973466e-18,-1.123309e-17,6.095605e-17,6.706818e-17,-1.850157e-17,-4.410641e-17,-1.371098e-17,1.321540e-18,2.180542e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,

--- Соединяем все результаты в одну таблицу ---

In [21]:
data = pd.concat((data_num, data_nonbin, data[binary_col], data['hospital_death']), axis = 1)
data.describe()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,heart_rate_apache,intubated_apache,map_apache,resprate_apache,temp_apache,ventilated_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_glucose_min,d1_potassium_max,d1_potassium_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,icu_admit_source_Accident & Emergency,icu_admit_source_Floor,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,apache_3j_bodysystem_Cardiovascular,apache_3j_bodysystem_Gastrointestinal,apache_3j_bodysystem_Genitourinary,apache_3j_bodysystem_Gynecological,apache_3j_bodysystem_Hematological,apache_3j_bodysystem_Metabolic,apache_3j_bodysystem_Musculoskeletal/Skin,apache_3j_bodysystem_Neurological,apache_3j_bodysystem_Respiratory,apache_3j_bodysystem_Sepsis,apache_3j_bodysystem_Trauma,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses,gender,hospital_death
count,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,8.602600e+04,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.000000,86026.00000,86026.000000,86026.000000,86026.000000,860

--- Подготовливаем данные к непосредственному применению ---

In [23]:
# Разделили признаки и результат
X = data.drop('hospital_death', axis = 1)
y = data['hospital_death']

# Разделили данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

print(X_train.shape[0])
print(X_test.shape[0])

64519
21507


--- Метод ближайших соседей ---

In [24]:
# Пробный запуск
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train, y_train)

# Погрешность на обучающей выборке
err_test = 1 - knn.score(X_test, y_test)
print(err_test)

# Погрешность на тренировочной выборке
err_train = 1 - knn.score(X_train, y_train)
print(err_train)

0.08313572325289442
0.07484616934546418


Вывод: метод k ближайших соседей (k = 10) дал довольно хороший результат. Погрешности на тренировочной и обучающей выборках
не сильно отличаются и сравнительно малы, значит переобучения и недообучения не произошло. Задача классификации решается достаточно точно

In [25]:
# Ищем лучшее количество соседей из 5 вариантов, представленных ниже
nnb = [4, 8, 10, 15, 19]
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid = {'n_neighbors': nnb})
grid.fit(X_train, y_train)

best_cv_err = 1 - grid.best_score_
best_n_neighbors = grid.best_estimator_.n_neighbors
print(best_cv_err, best_n_neighbors)

knn = KNeighborsClassifier(n_neighbors = best_n_neighbors)
knn.fit(X_train, y_train)

err_train = 1 - knn.score(X_train, y_train)
err_test = 1 - knn.score(X_test, y_test)
print(err_train, err_test)

0.07847302532382039 15
0.07537314589500765 0.08141535314083792


--- Random Forest ---

In [26]:
rf = ensemble.RandomForestClassifier(n_estimators = 1000)
rf.fit(X_train, y_train)

err_train = 1 - rf.score(X_train, y_train)
err_test = 1 - rf.score(X_test, y_test)
print(err_train, err_test)

0.0 0.07416190077649143


--- Ada Boost ---

In [27]:
ada = ensemble.AdaBoostClassifier(n_estimators = 1000)
ada.fit(X_train, y_train)

err_train = 1 - ada.score(X_train, y_train)
err_test = 1 - ada.score(X_test, y_test)
print(err_train, err_test)

0.06856894868178365 0.07346445343376573


--- Extremely Randomized Trees ---

In [28]:
ert = ensemble.ExtraTreesClassifier(n_estimators = 1000)
ert.fit(X_train, y_train)

err_train = 1 - ert.score(X_train, y_train)
err_test = 1 - ert.score(X_test, y_test)
print(err_train, err_test)

0.0 0.07537080950388242


Общие выводы: Удалось построить модель, которая с достаточной степенью точности способна спрогнозировать летальность госпитализированного в зависимости от причин госпитализации.
Лучший результат дал метод Ada Boost